# Mouse Atlas Data

In [1]:
import pandas as pd

In [2]:
import sys
import importlib
import himc_helper_functions as hf
importlib.reload(hf)
hf.get_version()

0.12.3 cleaning vdj code


In [3]:
import pandas as pd
from copy import deepcopy
from clustergrammer2 import net
df = {}

>> clustergrammer2 backend version 0.5.6


In [4]:
df['gex'] = pd.read_csv('data/primary/allen_brain_atlas/genes_counts.csv', index_col=0)
df['gex'].shape

(24057, 1679)

In [5]:
df['gex'].head()

,A01101401,A01101402,A01101403,A01101404,A01101405,A01101406,A01101407,A01101408,A02271433,A02271434,...,Y11041422,Y11041423,Y11041424,Y11041425,Z11041426,Z11041427,Z11041428,Z11041429,Z11041430,Z11041431
0610005C13Rik,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
0610007C21Rik,992.00,2287.02,491.78,1932.0,1425.0,130.03,2110.02,955.0,326.0,933.0,...,379.0,397.0,654.0,172.0,0.0,443.0,1457.0,4.0,2.08,2966.0
0610007L01Rik,2.57,177.00,0.00,1.0,2.0,3.00,3040.99,101.0,0.0,1042.0,...,0.0,411.0,471.0,692.0,0.0,0.0,0.0,12.0,115.00,2.0
0610007N19Rik,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
0610007P08Rik,0.00,0.00,0.00,0.0,0.0,0.00,17.03,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1014.00,0.0


In [6]:
df_meta = pd.read_csv('data/primary/allen_brain_atlas/cell_metadata.csv')
df_meta.index = df['gex'].columns.tolist()
df_meta.shape

(1679, 16)

In [7]:
df_meta.columns.tolist()

['long_name',
 'cre',
 'collection_date',
 'sequencing_type',
 'total_reads',
 'all_mapped_percent',
 'mRNA_percent',
 'genome_percent',
 'ercc_percent',
 'tdt_permillion',
 'major_class',
 'sub_class',
 'major_dissection',
 'layer_dissectoin',
 'color_code',
 'short_name']

In [8]:
df_meta['major_class'].value_counts()

Inhibitory     761
Excitatory     756
Glia           138
Endothelial     21
Unknown          3
Name: major_class, dtype: int64

In [9]:
df_meta['sub_class'].value_counts().shape

(32,)

In [10]:
df = hf.ini_meta_cell(df=df)

In [11]:
df = hf.filter_ribo_mito_from_gex(df)

keep_genes = [x for x in df['gex'].index.tolist() if 'mt_' not in x]
df['gex'] = df['gex'].loc[keep_genes]
df['meta_cell'].head()

,gex-umi-sum,num_expressed_genes,gex-ribo-avg,gex-mito-avg
A01101401,14030202.14,8266.40,1461.000686,0.0
A01101402,4803366.44,5954.36,638.585000,0.0
A01101403,3073418.37,6276.43,293.652549,0.0
A01101404,12192899.59,8312.21,1527.811176,13.0
A01101405,13477232.54,8086.26,1589.521176,0.0


In [12]:
df['gex'].shape

(23856, 1679)

In [13]:
df['meta_cell'] = pd.concat([df['meta_cell'], df_meta], axis=1)
df['meta_cell'].shape

(1679, 20)

In [14]:
df['gex-cat'] = deepcopy(net.umi_norm(df['gex']))
cols = df['gex-cat'].columns

add_cat_list = ['major_class', 'sub_class']

new_cols = hf.add_cats_from_meta(add_cat_list=add_cat_list, df_meta=df_meta, barcodes=cols)
df['gex-cat'].columns = new_cols

In [15]:
df['gex-cat'].columns.tolist()[0]

('A01101401', 'major_class: Inhibitory', 'sub_class: Vip')

In [16]:
df['gex-cat-umi'] = net.umi_norm(df['gex-cat'])

In [17]:
net.load_df(df['gex-cat-umi'])
net.filter_N_top(N_top=100, inst_rc='row', rank_type='var')
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "Malat1", "ini": 100, "clust": 59, "rank": 41, "rankvar": 44, "…

In [18]:
list_cats = [x[2] for x in df['gex-cat-umi'].columns.tolist()]
len(pd.Series(list_cats).value_counts())

33

In [19]:
num_top_dims = 100
df_sig, keep_genes_dict, df_gene_pval, all_fold_info = net.generate_signatures(df_ini=df['gex-cat-umi'], 
                                                                               category_level='sub_class', 
                                                                               num_top_dims=num_top_dims)
df_sig.shape

(1513, 33)

In [20]:
net.load_df(df_sig)
net.normalize(axis='row', norm_type='zscore')
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "0610011F06Rik", "ini": 1513, "clust": 869, "rank": 1491, "rank…

In [21]:
df_sig.columns = [x[0] for x in df_sig.columns.tolist()]

In [22]:
df_sig.columns.tolist()

['Astrocyte',
 'Chodl',
 'Endothelial',
 'L2/3',
 'L2/3_Syt10',
 'L4',
 'L5',
 'L5_Chrna6',
 'L5a',
 'L5a1',
 'L5a2',
 'L5b',
 'L5b1',
 'L5b2',
 'L6a',
 'L6a1',
 'L6a2',
 'L6b',
 'Microglia',
 'Ndnf',
 'OPC',
 'Oligodendrocyte',
 'Pvalb',
 'Sst',
 'Sst_Cbln4',
 'Sst_Chodl',
 'Sst_Clstn2',
 'Sst_Nmbr',
 'Sst_Nr2f2',
 'Th',
 'Unknown',
 'Vip',
 'nan']

In [23]:
df_sig.to_csv('data/primary/allen_brain_atlas/sig_sub_class.csv')